# DATA

In [ ]:
# Core Libraries
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import joblib

# ML Libraries
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report

# Deep Learning
import torch
from ultralytics import YOLO

In [ ]:

class ThermalPoseProcessor:
    def __init__(self, dataset_path, model_path):
        """
        Initialize the thermal pose processing pipeline.
        
        Args:
            dataset_path (str): Path to the dataset directory
            model_path (str): Path to the trained YOLOv8-Pose model
        """
        self.dataset_path = "artifacts/OPEN_THERMAL_IMAGE/train"
        self.model_path = "artifacts/model/openthermalpose"
        self.pose_model = None
        self.classifier = None
        self.label_encoder = LabelEncoder()
        
    def load_pose_model(self):
        """Load the YOLOv8-Pose model"""
        self.pose_model = YOLO(self.model_path)
        print("Pose model loaded successfully.")
        
    def load_and_prepare_data(self, test_size=0.2, val_size=0.1, random_state=42):
        """
        Load the dataset and prepare train/val/test splits.
        
        Args:
            test_size (float): Proportion of data for test set
            val_size (float): Proportion of training data for validation set
            random_state (int): Random seed for reproducibility
            
        Returns:
            dict: Dictionary containing the splits (X_train, X_val, X_test, y_train, y_val, y_test)
        """
        # Load activity labels
        labels_path = os.path.join(self.dataset_path, 'activity_labels.csv')
        df = pd.read_csv(labels_path)
        
        # Extract image paths and labels
        image_paths = [os.path.join(self.dataset_path, 'train', img_name) 
                       for img_name in df['image_name']]
        labels = df['activity_label'].values
        
        # Encode labels
        y = self.label_encoder.fit_transform(labels)
        
        # Split into train+val and test
        X_train_val, X_test, y_train_val, y_test = train_test_split(
            image_paths, y, test_size=test_size, random_state=random_state, stratify=y)
        
        # Split train_val into train and val
        X_train, X_val, y_train, y_val = train_test_split(
            X_train_val, y_train_val, test_size=val_size/(1-test_size), 
            random_state=random_state, stratify=y_train_val)
            
        return {
            'X_train': X_train,
            'X_val': X_val,
            'X_test': X_test,
            'y_train': y_train,
            'y_val': y_val,
            'y_test': y_test
        }
    
    def extract_keypoints(self, image_paths, batch_size=8):
        """
        Extract pose keypoints from images using YOLOv8-Pose.
        
        Args:
            image_paths (list): List of image paths to process
            batch_size (int): Batch size for processing
            
        Returns:
            np.ndarray: Array of keypoint vectors (n_samples, 51)
        """
        if not self.pose_model:
            raise ValueError("Pose model not loaded. Call load_pose_model() first.")
            
        keypoint_vectors = []
        
        # Process images in batches
        for i in tqdm(range(0, len(image_paths), batch_size), desc="Extracting keypoints"):
            batch_paths = image_paths[i:i+batch_size]
            results = self.pose_model(batch_paths)
            
            for result in results:
                if result.keypoints is not None and len(result.keypoints.xy) > 0:
                    # Get the first person's keypoints (assuming single person per image)
                    kpts = result.keypoints.xy[0].cpu().numpy().flatten()
                    
                    # If fewer than 17 keypoints, pad with zeros
                    if len(kpts) < 51:  # 17 keypoints * 3 (x,y,confidence)
                        kpts = np.pad(kpts, (0, 51 - len(kpts)))
                    keypoint_vectors.append(kpts)
                else:
                    # If no keypoints detected, use zero vector
                    keypoint_vectors.append(np.zeros(51))
                    
        return np.array(keypoint_vectors)
    
    def train_classifier(self, X_train, y_train, classifier_type='mlp'):
        """
        Train a classifier on the extracted keypoint features.
        
        Args:
            X_train (np.ndarray): Training features
            y_train (np.ndarray): Training labels
            classifier_type (str): Type of classifier ('mlp', 'random_forest')
        """
        if classifier_type == 'mlp':
            self.classifier = MLPClassifier(
                hidden_layer_sizes=(128, 64), 
                activation='relu',
                early_stopping=True,
                random_state=42
            )
        elif classifier_type == 'random_forest':
            self.classifier = RandomForestClassifier(
                n_estimators=100,
                max_depth=10,
                random_state=42
            )
        else:
            raise ValueError(f"Unknown classifier type: {classifier_type}")
            
        self.classifier.fit(X_train, y_train)
        print(f"{classifier_type.upper()} classifier trained successfully.")
    
    def evaluate_classifier(self, X, y, split_name='validation'):
        """
        Evaluate the classifier on given data.
        
        Args:
            X (np.ndarray): Features to evaluate
            y (np.ndarray): Ground truth labels
            split_name (str): Name of the split for display purposes
        """
        if not self.classifier:
            raise ValueError("Classifier not trained. Call train_classifier() first.")
            
        y_pred = self.classifier.predict(X)
        report = classification_report(
            y, 
            y_pred, 
            target_names=self.label_encoder.classes_
        )
        
        print(f"\n{classifier_type.upper()} Performance on {split_name} set:")
        print(report)
        return report
    
    def save_pipeline(self, output_dir):
        """Save the trained pipeline components"""
        os.makedirs(output_dir, exist_ok=True)
        joblib.dump(self.classifier, os.path.join(output_dir, 'classifier.joblib'))
        joblib.dump(self.label_encoder, os.path.join(output_dir, 'label_encoder.joblib'))
        print(f"Pipeline components saved to {output_dir}")
    
    def load_pipeline(self, output_dir):
        """Load the trained pipeline components"""
        self.classifier = joblib.load(os.path.join(output_dir, 'classifier.joblib'))
        self.label_encoder = joblib.load(os.path.join(output_dir, 'label_encoder.joblib'))
        print(f"Pipeline components loaded from {output_dir}")

def run_pipeline(dataset_path, model_path, output_dir='saved_models', classifier_type='mlp'):
    """
    Run the complete thermal pose activity recognition pipeline.
    
    Args:
        dataset_path (str): Path to the dataset directory
        model_path (str): Path to the trained YOLOv8-Pose model
        output_dir (str): Directory to save trained models
        classifier_type (str): Type of classifier to use ('mlp' or 'random_forest')
    """
    # Initialize the processor
    processor = ThermalPoseProcessor(dataset_path, model_path)
    
    # Step 1: Load pose model
    processor.load_pose_model()
    
    # Step 2: Prepare data splits
    splits = processor.load_and_prepare_data()
    
    # Step 3: Extract keypoint features
    print("\nExtracting features from training set...")
    X_train_features = processor.extract_keypoints(splits['X_train'])
    
    print("\nExtracting features from validation set...")
    X_val_features = processor.extract_keypoints(splits['X_val'])
    
    print("\nExtracting features from test set...")
    X_test_features = processor.extract_keypoints(splits['X_test'])
    
    # Step 4: Train classifier
    processor.train_classifier(X_train_features, splits['y_train'], classifier_type)
    
    # Step 5: Evaluate classifier
    _ = processor.evaluate_classifier(X_train_features, splits['y_train'], 'training')
    _ = processor.evaluate_classifier(X_val_features, splits['y_val'], 'validation')
    test_report = processor.evaluate_classifier(X_test_features, splits['y_test'], 'test')
    
    # Step 6: Save pipeline
    processor.save_pipeline(output_dir)
    
    return test_report

if __name__ == "__main__":
    # Example usage
    dataset_path =  "artifacts/OPEN_THERMAL_IMAGE/train"
    model_path = "artifacts/model/openthermalpose/"
    
    # Run with MLP classifier
    print("Running pipeline with MLP classifier...")
    mlp_report = run_pipeline(dataset_path, model_path, classifier_type='mlp')
    
    # Run with Random Forest classifier
    print("\nRunning pipeline with Random Forest classifier...")
    rf_report = run_pipeline(dataset_path, model_path, classifier_type='random_forest')

# LSTM MODEL

In [ ]:
import os
os.chdir("../")



In [ ]:
%pwd

In [7]:
# DL LSTM Classifier Training Pipeline
# Author: Priyam

from pathlib import Path
from dataclasses import dataclass
import numpy as np
import pandas as pd
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score
from sklearn.utils.multiclass import unique_labels
import joblib
from tqdm import tqdm


# ------------------------------------------------------------
# Config
# ------------------------------------------------------------
@dataclass
class Config:
    feature_dir: Path = Path("artifacts/pose_features")
    model_dir: Path = Path("artifacts/classifiers")
    report_dir: Path = Path("artifacts/classifiers/eval_reports")
    label_encoder_path: Path = model_dir / "label_encoder.pkl"

    batch_size: int = 32
    num_epochs: int = 20
    hidden_dim: int = 128
    learning_rate: float = 1e-3
    device: str = "cuda" if torch.cuda.is_available() else "cpu"

    def resolve_paths(self):
        for attr in ['feature_dir', 'model_dir', 'report_dir']:
            p = getattr(self, attr)
            p.mkdir(parents=True, exist_ok=True)
        return self


# ------------------------------------------------------------
# Dataset
# ------------------------------------------------------------
class PoseDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)

    def __len__(self): return len(self.X)
    def __getitem__(self, idx): return self.X[idx], self.y[idx]


# ------------------------------------------------------------
# LSTM Classifier
# ------------------------------------------------------------
class LSTMClassifier(nn.Module):
    def __init__(self, input_dim=51, hidden_dim=128, num_classes=10):
        super().__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, num_classes)

    def forward(self, x):
        # Assuming (batch_size, seq_len, input_dim)
        _, (h_n, _) = self.lstm(x)
        return self.fc(h_n[-1])


# ------------------------------------------------------------
# Training & Evaluation
# ------------------------------------------------------------
def train_model(cfg: Config):
    cfg.resolve_paths()

    # Load features
    X_train = np.load(cfg.feature_dir / "train.npy")
    X_test = np.load(cfg.feature_dir / "test.npy")
    y_train = pd.read_csv(cfg.feature_dir / "train_labels.csv")["label"].values
    y_test = pd.read_csv(cfg.feature_dir / "test_labels.csv")["label"].values

    # Label encoding
    le = LabelEncoder()
    y_train_enc = le.fit_transform(y_train)
    y_test_enc = le.transform(y_test)
    joblib.dump(le, cfg.label_encoder_path)

    # Dataset and loader
    X_train_seq = X_train[:, None, :]  # shape: (N, 1, 51)
    X_test_seq = X_test[:, None, :]

    train_ds = PoseDataset(X_train_seq, y_train_enc)
    test_ds = PoseDataset(X_test_seq, y_test_enc)
    train_loader = DataLoader(train_ds, batch_size=cfg.batch_size, shuffle=True)
    test_loader = DataLoader(test_ds, batch_size=cfg.batch_size)

    # Model
    model = LSTMClassifier(input_dim=51, hidden_dim=cfg.hidden_dim, num_classes=len(le.classes_))
    model.to(cfg.device)
    optimizer = torch.optim.Adam(model.parameters(), lr=cfg.learning_rate)
    loss_fn = nn.CrossEntropyLoss()

    # Training
    model.train()
    for epoch in range(cfg.num_epochs):
        total_loss = 0
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(cfg.device), y_batch.to(cfg.device)
            optimizer.zero_grad()
            logits = model(X_batch)
            loss = loss_fn(logits, y_batch)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        print(f"Epoch {epoch+1}/{cfg.num_epochs} - Loss: {total_loss/len(train_loader):.4f}")

    # Save model
    torch.save(model.state_dict(), cfg.model_dir / "lstm_classifier.pt")
    print(f"[✓] Saved model to {cfg.model_dir / 'lstm_classifier.pt'}")

    # Evaluation
    model.eval()
    y_true, y_pred = [], []
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            X_batch = X_batch.to(cfg.device)
            logits = model(X_batch)
            preds = torch.argmax(logits, dim=1).cpu()
            y_true.extend(y_batch.numpy())
            y_pred.extend(preds.numpy())

    acc = accuracy_score(y_true, y_pred)

    # Fix for mismatched label sizes
    present_labels = unique_labels(y_true, y_pred)
    report = classification_report(
        y_true, y_pred,
        labels=present_labels,
        target_names=le.inverse_transform(present_labels)
    )

    print(f"\nTest Accuracy: {acc:.4f}")
    print(report)

    with open(cfg.report_dir / "lstm_evaluation.txt", "w") as f:
        f.write(f"Accuracy: {acc:.4f}\n")
        f.write(report)

    print(f"[✓] Saved report to {cfg.report_dir / 'lstm_evaluation.txt'}")


# ------------------------------------------------------------
# Entry point
# ------------------------------------------------------------
if __name__ == "__main__":
    cfg = Config()
    train_model(cfg)


Epoch 1/20 - Loss: 1.3112
Epoch 2/20 - Loss: 1.1152
Epoch 3/20 - Loss: 1.0734
Epoch 4/20 - Loss: 1.0367
Epoch 5/20 - Loss: 1.0150
Epoch 6/20 - Loss: 0.9993
Epoch 7/20 - Loss: 0.9875
Epoch 8/20 - Loss: 0.9753
Epoch 9/20 - Loss: 0.9682
Epoch 10/20 - Loss: 0.9596
Epoch 11/20 - Loss: 0.9570
Epoch 12/20 - Loss: 0.9492
Epoch 13/20 - Loss: 0.9419
Epoch 14/20 - Loss: 0.9418
Epoch 15/20 - Loss: 0.9351
Epoch 16/20 - Loss: 0.9285
Epoch 17/20 - Loss: 0.9232
Epoch 18/20 - Loss: 0.9190
Epoch 19/20 - Loss: 0.9155
Epoch 20/20 - Loss: 0.9146
[✓] Saved model to artifacts/classifiers/lstm_classifier.pt

Test Accuracy: 0.8391
                     precision    recall  f1-score   support

EXERCISE_BODY_SWING       0.64      0.90      0.75       185
   LOOKING_STRAIGHT       1.00      0.99      0.99      1081
   SITTING_STANDING       0.00      0.00      0.00        15
           STANDING       0.00      0.00      0.00       129
           fighting       0.08      0.03      0.04        36
          gesturing

/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
